### Задача: 
реализовать механизм проверки предприятия на банкротство, используя ранее обученную модель для прогнозирования вероятности банкротства

In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
# загрузим модель предсказания банкротства из файла 'model.sav'
# (модель создается и записывается в ноутбуке 'bankruptcy forecast.ipynb')
model = pickle.load(open('model.sav', 'rb'))

In [3]:
def analysis_bankrupt(model, csv_filename, pkl_filename):
    '''
    Функция выполняет анализ предприятия на вероятность банкротства на основании созданной ранее модели.
    
    В ходе работы данной функции загружаются данные из указанных файлов и проводится подготовка данных,
    которая полностью соответствует подготовке данных при создании и обучении модели 
    (см. ноутбук 'bankruptcy forecast.ipynb').
    
    Вход: 
        model - обученная модель
        csv_filename - данные предприятия об обязательствах и активах
        pkl_filename - данные предприятия об участии в арбитражах
    Выход: 1 - есть вероятность банкротства данного предприятия, 0 - нет
    '''

    # загрузим данные об активах и задолженности компании из csv-файла в датафрейм 'df_accounts'
    df_account = pd.read_csv(csv_filename, sep=',')
    df_account.rename(columns={'Unnamed: 0': 'id',
                               'long_term_liabilities_fiscal_year': 'long_liabilities',
                               'short_term_liabilities_fiscal_year': 'short_liabilities',
                               'balance_assets_fiscal_year': 'balance_assets'}, inplace=True)
    df_account.set_index('id', inplace=True)
    df_account.fillna(0, inplace=True)
    df_account = df_account.astype({'year': 'int16',
                                    'short_liabilities': 'float64',
                                    'balance_assets': 'float64',
                                    'okei': 'int16'})

    # загрузим данные из pkl-файла в датафрейм 'df_court_cases'
    # pkl-файл хранит сведения об участии предприятия в арбитражных делах
    # поле case_sides.type принимает значения: 
    # 0 - 'истец', 1 - 'ответчик', 2 - 'свидетель', 3 - 'конкурсный управляющий' 
    # (type 3 был выявлен при анализе pkl-файлов)
    # для подсчета кол-ва участий по каждому типу стороны дела будем пользоваться словарем
    df_court_cases = pd.DataFrame(columns=['inn', 'year', 0, 1, 2, 3])

    with open(pkl_filename, 'rb') as f:
        data = pickle.load(f)

    inn = data['inn']

    court_case = dict()    
    for i in range(len(data['cases_list'])):
        year = pd.to_datetime(data['cases_list'][i]['caseDate']).year
        if year not in court_case:
            court_case[year] = {'inn': inn, 'year': year, 0: 0, 1: 0, 2: 0, 3: 0}
        count_case_sides = len(data['cases_list'][i]['case_sides'])
        for j in range(count_case_sides):
            if data['cases_list'][i]['case_sides'][j]['INN'] == str(inn):
                court_case[year][data['cases_list'][i]['case_sides'][j]['type']] += 1

    court_case_list = [x for x in court_case.values()]

    df_court_cases = df_court_cases.append(court_case_list, ignore_index=True)

    # дадим 'осмысленные' названия столбцам и приведем в порядок типы данных
    df_court_cases.rename(columns={0: 'applicant', 1: 'responder', 2: 'witness', 3: 'bankrupt_trustee'}, inplace=True)
    df_court_cases = df_court_cases.astype({'inn': 'int64',
                                            'year': 'int16',
                                            'applicant': 'int16',
                                            'responder': 'int16',
                                            'witness': 'int16',
                                            'bankrupt_trustee': 'int16'})

    # данные об обязательствах и баланс приведем к тыс.руб.
    def calc(x, y):
        if y == 383:
            x /= 1000
        elif y == 385:
            x *= 1000
        return x

    df_account['long_liabilities_th'] = df_account.apply(lambda x: calc(x['long_liabilities'], x['okei']), axis=1)
    df_account['short_liabilities_th'] = df_account.apply(lambda x: calc(x['short_liabilities'], x['okei']), axis=1)
    df_account['balance_assets_th'] = df_account.apply(lambda x: calc(x['balance_assets'], x['okei']), axis=1)

    # соберем все сведения о предприятии в единый датафрейм 'df_firms'
    df_firm = pd.merge(df_account, df_court_cases, on=['inn', 'year'], how='outer') 
    df_firm.drop(['long_liabilities', 'short_liabilities', 'balance_assets', 'okei'], axis=1, inplace=True)
    df_firm.fillna(0, inplace=True)

    # приведем в порядок типы данных
    df_firm = df_firm.astype({'year': 'int16',
                              'long_liabilities_th': 'float64',
                              'short_liabilities_th': 'float64',
                              'balance_assets_th': 'float64',
                              'applicant': 'int16',
                              'responder': 'int16',
                              'witness': 'int16',
                              'bankrupt_trustee': 'int16'})

    # сгруппируем даные о предприятии и просуммируем кол-во сведений об участии в арбитражах,
    # а данные об обязательствах и баланс возьмем по последнему году
    # добавим еще один признак - кол-во лет предприятию
    # (в данном случае кол-во наблюдений, т.к. у нас нет информации о дате создания предприятия)
    df_firm_agg = df_firm.sort_values(by=['year']).groupby('inn', as_index=False).\
        agg(year=('year', 'last'),
            count_years=('year', 'count'),
            long_liabilities_th=('long_liabilities_th', 'last'),
            short_liabilities_th=('short_liabilities_th', 'last'),
            balance_assets_th=('balance_assets_th', 'last'),
            applicant=('applicant', 'sum'),
            responder=('responder', 'sum'),
            witness=('witness', 'sum'),
            bankrupt_trustee=('bankrupt_trustee', 'sum')
           )

    df_firm_agg.drop(['year'], axis=1, inplace=True)
    df_firm_agg.set_index('inn', inplace=True)

    # сделаем прогноз для указанного предприятия
    y_pred = model.predict(df_firm_agg)
    
    return y_pred[0]

In [4]:
csv_filename = 'test/1302000640.csv'
pkl_filename = 'test/1302000640_b.pkl'

In [5]:
res = analysis_bankrupt(model, csv_filename, pkl_filename)
res_str = 'есть' if res == 1 else 'нет'
print('Вероятность банкротства:', res_str)

Вероятность банкротства: есть


In [6]:
csv_filename = 'test/9102060446.csv'
pkl_filename = 'test/9102060446.pkl'

In [7]:
res = analysis_bankrupt(model, csv_filename, pkl_filename)
res_str = 'есть' if res == 1 else 'нет'
print('Вероятность банкротства:', res_str)

Вероятность банкротства: нет
